<a href="https://colab.research.google.com/github/RahafDayoub4/agents/blob/main/pdfreader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 61.4 MB/s eta 0:00:00


In [3]:
!pip install smolagents -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.0/134.0 kB 11.5 MB/s eta 0:00:00


In [4]:
from google.colab import files
uploaded = files.upload()


Saving agentTest.pdf to agentTest.pdf


In [15]:
import os
print(os.listdir())


['.config', 'agentTest.pdf', 'sample_data']


In [6]:
from smolagents import tool
import fitz  # PyMuPDF

@tool
def extract_text_from_pdf(file_path: str) -> str:
    """
    Extract all text from a PDF file.
    Args:
        file_path (str): Path to the PDF file.
    Returns:
        str: Extracted text.
    """
    try:
        doc = fitz.open(file_path)
        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()
        return text if text.strip() else "No text found in the PDF."
    except Exception as e:
        return f"Error loading PDF: {e}"


In [7]:
from smolagents import CodeAgent, tool
from transformers import pipeline

# تهيئة موديل التلخيص محلياً مرة واحدة
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

@tool
def summarize_text_local(text: str) -> str:
    """
    Summarizes long text using a local model.

    Args:
        text (str): Long text.

    Returns:
        str: Summary of the text.
    """
    summary = summarizer(text, max_length=150, min_length=50, do_sample=False)
    return summary[0]['summary_text']

@tool
def split_and_summarize_local(text: str, max_chunk_words: int = 500) -> str:
    """
    Splits a long text into chunks, summarizes each chunk,
    then merges partial summaries and summarizes them again.

    Args:
        text (str): Long text.
        max_chunk_words (int): Maximum number of words per chunk.

    Returns:
        str: Final summary of the entire text.
    """
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_chunk_words):
        chunk = " ".join(words[i:i+max_chunk_words])
        chunks.append(chunk)

    partial_summaries = []
    for chunk in chunks:
        partial_summary = summarize_text_local(chunk)
        partial_summaries.append(partial_summary)

    combined_summary_text = " ".join(partial_summaries)
    final_summary = summarize_text_local(combined_summary_text)

    return final_summary

# ممكن تضيف أدوات أخرى هنا لو حبيت (مثلاً extract_text_from_pdf)

# تعريف الوكيل بدون موديل API
agent = CodeAgent(
    tools=[split_and_summarize_local, summarize_text_local],  # استعمل أدوات محلية فقط
    model=None  # لا تستخدم موديل API خارجي
)

# # تشغيل الوكيل (مثال)
# command = "Split and summarize the following text: [أدخل نص طويل هنا]"
# result = agent.run(command)
# print("Agent Output:", result)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [12]:
from huggingface_hub import login

login()


In [8]:
from smolagents import tool

@tool
def generate_tweet(summary: str) -> str:
    """
    Generates a catchy tweet from a paper summary.
    Args:
        summary (str): The summary of the paper.
    Returns:
        str: Tweet-ready post.
    """
    return f"🚀 New Research Highlight:\n{summary}\n#AI #Research #Science"


In [12]:
from smolagents import CodeAgent, InferenceClientModel

agent = CodeAgent(
    tools=[extract_text_from_pdf, split_and_summarize_local],  # استخدم هنا دالة التلخيص المحلية
    model=InferenceClientModel()  # ممكن تستخدم موديلك الحالي أو تعدل حسب الحاجة
)

# تشغيل الوكيل
agent.run("Extract the PDF from 'agentTest.pdf', split and summarize it.")


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ split_and_summarize_local(extract_text_from_pdf('agentTest.pdf'))                                               │
│                                                                                                                 │
╰─ NoneType -  ───────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
'NoneType' object has no attribute 'generate'

[Step 1: Duration 0.00 seconds]

AgentGenerationError: Error in generating model output:
'NoneType' object has no attribute 'generate'